### Тестовое задание

In [ ]:
rounded_desc_act = desc_act_base.select(
    *[
        F.when(F.col(c).cast("double").isNotNull(), F.round(F.col(c).cast("double"), 2))
         .otherwise(F.col(c))
         .alias(c)
        for c in desc_act_base.columns
    ]
)

rounded_desc_act.show(truncate=False)

In [ ]:
# Добавляем год и месяц
actions = actions.withColumn("year", F.year("date_from")).withColumn("year_month", F.date_format("date_from", "yyyy-MM"))

# Считаем количество промо
promo_cnt_month = actions.groupBy("year_month").agg(F.count("*").alias("count")).orderBy("year_month")
promo_cnt_year = actions.groupBy("year").agg(F.count("*").alias("count")).orderBy("year")


# Подготовка к визуализации
promo_cnt_month = promo_cnt_month.toPandas()
promo_cnt_month["year_month"] = pd.to_datetime(promo_cnt_month['year_month'])
promo_cnt_month = promo_cnt_month.sort_values('year_month')

promo_cnt_year = promo_cnt_year.toPandas()

# Строим график
fig, axes = plt.subplots(2,1, figsize=(12,5), sharey=False)
axes[0].bar(promo_cnt_year["year"].astype(str), promo_cnt_year["count"]) 
axes[1].bar(promo_cnt_month["year_month"], promo_cnt_month["count"], width=20)
axes[1].set_xlabel("Период (год/месяц)")

plt.suptitle("Количество промо по годам/месяцам", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
print("Временной период истории продаж")
actions.select(F.min("date_from"), F.max("date_to")).show()
# Можно ещё посмотреть на распределение количества акций по годам
print("Распределение количества акций по годам")
actions.withColumn("year", F.year("date_from")) \
       .groupBy("year").agg(F.countDistinct("promo_id").alias("promos")).orderBy("year").show()
print("Распределение количества акций по месяцам")
actions.withColumn("year", F.year("date_from")).withColumn("month", F.month("date_from")) \
       .groupBy("year","month").agg(F.countDistinct("promo_id").alias("promos")).orderBy("year","month").show()

#### Вопрос №1.1:

В конце каждого месяца компания выдает премию для своих курьеров, средняя скорость доставки за прошедший месяц которых больше средней скорости среди всех курьеров. Сколько курьеров получили премию за июнь 2021 года.

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Test task").getOrCreate()

# Проверка успешного подключения
print("Spark Version:", spark.version)

Spark Version: 3.5.4


In [2]:
df1 = spark.read.parquet("couriers_orders.parquet")

In [3]:
df1.show(5)

+-------------------+----------+--------+--------+-----------+
|               date|courier_id|order_id|distance|travel_time|
+-------------------+----------+--------+--------+-----------+
|2021-07-12 00:00:00|        10|       1|     1.9|      36.17|
|2021-07-02 00:00:00|         3|       2|    3.98|      21.34|
|2021-04-15 00:00:00|         6|       3|    3.98|      43.33|
|2021-07-16 00:00:00|        10|       4|    2.85|      14.01|
|2021-06-11 00:00:00|        10|       5|    4.89|      32.09|
+-------------------+----------+--------+--------+-----------+
only showing top 5 rows



In [5]:
from pyspark.sql import functions as F
# добавим скорость, а также выделяем год и месяц
df1 = df1.withColumn("speed", F.col("distance")/F.col("travel_time"))
df1 = df1.withColumn("year", F.year("date")).withColumn("month", F.month("date"))
# считаем ср. скорость курьера
courier_avg_speed = df1.groupBy("year", "month", "courier_id").agg(F.avg("speed").alias("avg_speed"))
# считаем ср.скорость по всем курьерам
overall_avg_speed = courier_avg_speed.groupBy("year", "month").agg(F.avg("avg_speed").alias("overall_avg_speed"))
#соединяем для сравнения 
vs_speed = courier_avg_speed.join(
    overall_avg_speed,
    on = ["year", "month"],
    how = "inner"
)
#фильтруем тех курьеров, у которых средняя скорость была больше  
tmp = vs_speed.filter(F.col("avg_speed")>F.col("overall_avg_speed"))
# фильтруем конкретный месяц
result = tmp.filter(
    (F.col("year") == 2021) & (F.col("month") == 6)
).select("courier_id").distinct().count()

print("Количество курьеров с премией в июне 2021 года: ", result)

Количество курьеров с премией в июне 2021 года:  6


#### Вопрос №1.2 (используйте данные из предыдущего вопроса №1.1):

Компания хочет понять, насколько равномерно курьеры работают в течение месяца. Для этого нужно найти ID курьера с наибольшей разницей между максимальной и минимальной средней дневной скоростью в июне 2021 года.

In [11]:
df1 = spark.read.parquet("couriers_orders.parquet")
df1 = df1.withColumn("speed", F.col("distance")/F.col("travel_time"))
month = df1.filter(
    (F.col("date") >= "2021-06-01") & (F.col("date") <= "2021-07-01")
)
df_day = month.withColumn("day", F.to_date("date"))

daily_speed = df_day.groupBy("courier_id", "day").agg(
    F.avg("speed").alias("avg_daily_speed")
)

diff = daily_speed.groupBy("courier_id").agg(
    (F.max('avg_daily_speed') - F.min("avg_daily_speed")).alias("speed_diff")
)

result = diff.orderBy(F.col("speed_diff").desc()).limit(1)
result.show()

+----------+------------------+
|courier_id|        speed_diff|
+----------+------------------+
|         4|0.4133767777133771|
+----------+------------------+



У нас есть данные о покупках клиентов purchases.parquet. Проанализируйте интервалы времени между последовательными покупками для каждого клиента в наборе данных о покупках - напишите код для вычисления разницы в днях между текущей покупкой и предыдущей покупкой каждого клиента. Отобразите результат в новом столбце days_between_purchases.


#### Вопрос №2.1:

Какое количество NaN в столбце days_between_purchases?

In [12]:
df2 = spark.read.parquet("purchases.parquet")

In [13]:
df2.show(5)

+-----------+-------------------+
|customer_id|      purchase_date|
+-----------+-------------------+
|          2|2021-01-01 00:00:00|
|          7|2021-01-01 00:00:00|
|          7|2021-01-01 00:00:00|
|         11|2021-01-01 00:00:00|
|         21|2021-01-01 00:00:00|
+-----------+-------------------+
only showing top 5 rows



In [14]:
from pyspark.sql import functions as F
from pyspark.sql import Window
#оконная функция по каждому клиенту с сортировкой по дате покупки
w = Window.partitionBy("customer_id").orderBy("purchase_date")
#лагом в окне находим дату предыдущей покупки
df2 = df2.withColumn("prev_purchase_date", F.lag("purchase_date").over(w))
# находим разницу в днях между покупками
df2 = df2.withColumn("days_between_purchases", F.datediff(F.col("purchase_date"),F.col("prev_purchase_date")))
# считаем nan в столбце
nan_count = df2.filter(F.col("days_between_purchases").isNull()).count()
print("Количество NaN в столбце days_between_purchases: ", nan_count)

Количество NaN в столбце days_between_purchases:  50


#### Вопрос №2.2 (используйте данные из предыдущего вопроса №2.1):

У какого количества уникальных клиентов разница между текущей покупкой и предыдущей покупкой равна 20-ти дням?

In [16]:
result = df2.filter(F.col("days_between_purchases") == 20)
count_unique_cust = result.select("customer_id").distinct().count()
print("Количество уникальных клиентов с разницей между покупками 20 дней: ", count_unique_cust)

Количество уникальных клиентов с разницей между покупками 20 дней:  10
